In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
real_df = pd.read_csv("Data set 3 (99 KB) - winequality.csv")
df = real_df.copy(deep=True)

***

In [3]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [5]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [6]:
def corrmat():
    plt.figure(figsize=(12, 8))

    corr = df.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    masked_corr = corr.mask(mask)
    sns.heatmap(masked_corr, annot=True, annot_kws={"size": 13}, fmt='.2f') 

    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    plt.show()

In [7]:
df['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

total 1536 wines
=====
| predicting | minority | Best score: | Accuracy: |
| :-: | :-: | :-: | :-: |
| 10 bad / 1526 good | 0.65 | 0.993 | 0.997 |
| 63 bad / 1473 good | 4.1 | 0.959 | 0.966 |
| 744 bad / 855 good | 48.4 | 0.771 | 0.747 |
| 217 good / 1382  bad | 14.13 | 0.903 | 0.891 |
| 18 good / 1581  bad | 1.13 | 0.991 | 0.981 |

In [8]:
# import libraries
import timeit
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# create a binary target variable
df["quality_status"] = [1 if i >= 6 else 0 for i in df["quality"]]  # 1 means good

# features
X = df.drop(['quality', 'quality_status'], axis=1)
# target
y = df['quality_status']

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the data using standard scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# import models
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

models = [
    ("KNN", KNeighborsClassifier(), 
        {'n_neighbors': [3, 5, 7, 9, 11, 13, 15], 
         'weights': ['uniform', 'distance'],
         'algorithm' : ['auto', 'ball_tree','kd_tree'],
         'p': [1, 2, 3, 4]}),
    
    ("Decision Tree", DecisionTreeClassifier(), 
        {'max_depth': [None, 5, 10], 
         'min_samples_split': [2, 5, 10],
         'criterion': ['gini', 'entropy']}),
    
    
    ("Naive Bayes Bernoulli", BernoulliNB(), 
        {'alpha': [0.1, 1, 10], 
         'fit_prior': [True, False],
         'binarize': [0.0, 0.5, 1.0]}),
    
    ("Logistic Regression", LogisticRegression(), 
        {'max_iter' : [2000],
         'penalty' : ['l1', 'l2'],
         'C' : np.logspace(-4, 4, 20),
         'solver' : ['liblinear']}),

    ("SVM", SVC(), 
       {'C': [0.1, 1, 10, 100, 1000],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        # 'degree' : [2,3,4,5],
        'gamma': [1, 0.1, 0.01, 0.001, 0.0001]})
]

# create 2 empty dataframes to store the output
row_names = ['best accuracy', 'best_params', 'precision', 'recall', 'f1_score', 'confusion matrix']
beforeTuned = pd.DataFrame(index=['cv mean accuracy','training time'])
Tuned = pd.DataFrame(index=row_names)

# loop through each model and fit, predict, and score with and without using GridSearchCV
for name, model, param_grid in models:
    print(f"Now training {name} model")
    # start timer to calculate the time taken to cross validate
    startTimer = timeit.default_timer()
    
    '''before tuning the hyperparameters'''
    cv = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    
    # calculate the time taken to cross validate
    endTimer = timeit.default_timer() - startTimer
    
    # calculate and store the accuracy of each model in the output dataframe
    beforeTuned.loc['cv mean accuracy',name] = cv.mean()
    beforeTuned.loc['training time', name] = endTimer
    
    '''tuning the hyperparameters'''
    # create a GridSearchCV instance for the current model
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=True)
    
    # fit the model on the scaled training data
    grid_search.fit(X_train, y_train)
    
    # predict on the test data using the best estimator found by GridSearchCV
    y_pred = grid_search.best_estimator_.predict(X_test)
    
    # calculate and store the accuracy of each model in the output dataframe
    Tuned.loc['best accuracy', name] = round(grid_search.best_estimator_.score(X_test, y_test),2)
    Tuned.loc['best_params', name] = str(grid_search.best_params_)
    Tuned.loc['precision', name] = round(precision_score(y_test, y_pred),4)
    Tuned.loc['recall', name] = round(recall_score(y_test, y_pred),4)
    Tuned.loc['f1_score', name] = round(f1_score(y_test, y_pred),4)
    Tuned.loc['confusion matrix', name] = str(confusion_matrix(y_test,y_pred)).replace('\n', '')
    

Now training KNN model
Fitting 5 folds for each of 168 candidates, totalling 840 fits
Now training Decision Tree model
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Now training Naive Bayes Bernoulli model
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Now training Logistic Regression model
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Now training SVM model
Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [ ]:
combined = pd.concat([beforeTuned.T, Tuned.T], axis=1)
summary = combined.sort_values(by="best accuracy", ascending=False)
summary = summary[['best accuracy', 'cv mean accuracy', 'training time', 'precision', 'recall', 'f1_score', 'confusion matrix', 'training time', 'best_params']]
summary

In [ ]:
from sklearn.inspection import permutation_importance

# Assuming you have already defined X_train, y_train, X_test, y_test, summary and params

params = eval(summary['best_params'][0])
model = KNeighborsClassifier(**params).fit(X_train, y_train)

# Compute the permutation importance on the test set
result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=0)

# Get the feature names and importance scores
feat_names = X.columns
feat_importances = result.importances_mean

# Sort the features by importance in descending order
sorted_idx = feat_importances.argsort()[::-1]
feat_names = feat_names[sorted_idx]
feat_importances = feat_importances[sorted_idx]

# Plot the top 10 features in a bar chart
plt.barh(feat_names[:10], feat_importances[:10])
plt.xlabel('Permutation Importance')
plt.ylabel('Feature')
plt.show()
